> Developpé par Valentin Macé dans le cadre du stage de fin de 4ème année du cycle Ingénieur Informatique (Polytech Marseille) au LIS

![LIS](img/lis.png)

### Introduction
<br><br>
<font color='orange'>Le but de ce script est de permettre à un fichier textuel au format Semeval 2010 d'être transformé vers le format utilisé par Mo Yu en entrée du FCM (Gormley, Yu 2015).</font>

_Publication du FCM : https://www.cs.cmu.edu/~mgormley/papers/gormley+yu+dredze.emnlp.2015.pdf_
<br><br><br>
***
Format d'entrée : 

    4  "A misty <e1>ridge</e1> uprises from the <e2>surge</e2>."
    Other
 On y distingue une première ligne avec un index et une phrase contenant deux entités e1 et e2, unde deuxième ligne avec le type de la relation entre les entités
 <br><br><br>
 
 Format de sortie :

    Other	2	2	ridge	6	6	surge
    A	DT	0	0	0	misty	JJ	B-adj.all	0	0	ridge	NN	B-noun.object	0	0	uprises	NNS	I-noun.object	0	1	from	IN	0	0	1	the	DT	0	0	0	surge	NN	B-noun.event	0	0	.	.	0	0	0
    1	B-noun.object 0
    1	B-noun.event 0

On y distingue une première ligne constituée de la relation entre e1 et e2, de l'indice de début de la première entité suivi de son indice de fin, de la première entité elle-même ('ridge'), le tout suivi des mêmes éléments pour la seconde entité

Une deuxième ligne composée de la phrase ("A misty...") annotée grâce au SuperSense Tagger (SST), attention il se peut que cette ligne très longue apparaisse comme plusieurs lignes

Enfin les deux dernières lignes représentent, pour chaque entité : sa taille suivie du deuxième tag généré par le SST, suivi du troisième tag
<br><br><br>
<font color='orange'>Remarque</font>, le dernier tag pour chaque mot n'est pas généré par le SST, c'est un boléen qui indique si le mot fait partie du dependency path entre e1 et e2

Pour l'obtenir, on crée pour chaque phrase du corpus un graphe de dépendence non orienté, et on regarde pour chaque mot de la phrase si il se trouve sur le chemin le plus court entre e1 et e2
***
<br><br>
Une fois ce format généré, il suffit de le donner en entrée du FCM

*Implementation du FCM en C++ par Yu https://github.com/Gorov/FCM_nips_workshop*
<br><br>


#### Setup

In [1]:
import copy
import numpy as np
import subprocess
import networkx as nx
import spacy
from spacy import displacy
nlp = spacy.load('en_core_web_sm')

#### Fonction de vérification de la structure du corpus

In [6]:
# Vérifie toutes les lignes multiples de 3 (donc les phrases du corpus)
# Si le corpus est décalé ce sera aussi détecté
def check_corpus(file_path):
    file = open(file_path, "r")
    lines = file.readlines()
    corpus_ok = True
    for i in range(0, len(lines), 3):
        if('<e1>' not in lines[i] or '<e2>' not in lines[i] or '</e2>' not in lines[i] or '</e1>' not in lines[i]):
            print("There is a problem with the corpus at the line ",i+1)
            corpus_ok = False
    if(corpus_ok):
        print("The corpus seems ok")
    
check_corpus("notebook_input.txt")

The corpus seems ok


#### Fonction qui retourne le chemin de dépendance le plus court entre deux mots d'une phrase

In [5]:
# Renvoie le chemin de dépendances le plus court
# Input : la phrase, les deux mots à partir desquels on calcul le chemin
# Ouput : un tableau contenant les mots présents sur le chemin de dépendance
def shortest_dependency_path(doc, e1=None, e2=None):
    edges = []
    shortest_path = []
    # On crée une arête entre chaque noeud (token) et ses enfants
    for token in doc:
        for child in token.children:
            edges.append(('{0}'.format(token),
                          '{0}'.format(child)))
    # Création du graphe avec ces arêtes
    graph = nx.Graph(edges)
    try:
        if(e1 in graph.nodes and e2 in graph.nodes):
            # Calcul du chemin le plus court
            shortest_path = nx.shortest_path(graph, source=e1, target=e2)
    except nx.NetworkXNoPath:
        shortest_path = []
    return shortest_path

#### Ici commence le traitement du fichier
<br>
#### Traitement des mots gênants, espacement des balises d'entités et suppression des indexes pour les phrases
<br>
Input:

    699	"Now , <e2>administration</e2> <e1>officials</e1> in U.S. are "good" people"
Output:

    Now , <e2> administration </e2> <e1> officials </e1> in U.S. are "good" people

In [7]:
# On charge le fichier contenant le texte à traiter - Format Semeval 2010
file = open("notebook_input.txt", "r")
# lines contiendra toutes les lignes du fichier original, nettoyées
lines = file.readlines()

# On supprime les caractères genants dans les mots : 'a.m.' ou 'U.S' -> 'am' et 'US'
# pb -> reAce possède des 'U.S .' et pas 'U.S.' donc le dernier point n'est pas supprimé
# possibilité d'amélioration, ainsi que 'a.m' qui devient le verbe 'am'
for i in range(0, len(lines), 3):                        # Toutes les 3 lignes... donc chaque phrase
    line_split = lines[i].split()                        # split sépare la phrase en un tableau de mots
    for j in range(len(line_split)):
        if(line_split[j].find(',') > 0):                 # Si la virgule fait partie d'un mot
            line_split[j] = line_split[j].replace(',','')
        if(line_split[j].find('.') > 0):
            line_split[j] = line_split[j].replace('.','')
        if(line_split[j] == '.' and j != len(line_split)-2):
            line_split[j] = line_split[j].replace('.','')


    lines[i] = ' '.join(line_split) + '\n'               # join permet de reconstruire la ligne
    
# On espace les entités pour que le split à venir les détecte
# On enlève les guillemets inutiles encadrant les phrases
i = 0
while i < len(lines):
    #lines[i] = lines[i].replace('"', '')                # Ancienne version on supprime tous les "
    line_split = lines[i].split()
    for j in range(len(line_split)):
        if(j == 1 and '"' in line_split[j]):
            line_split[j] = line_split[j].replace('"','')
        if(j == len(line_split)-1 and '"' in line_split[j]):
            line_split[j] = line_split[j].replace('"','')
    lines[i] = ' '.join(line_split) + '\n'

    lines[i] = lines[i].replace('<e1>', '<e1> ')
    lines[i] = lines[i].replace('<e2>', '<e2> ')
    lines[i] = lines[i].replace('</e1>', ' </e1>')
    lines[i] = lines[i].replace('</e2>', ' </e2>')
    i+=1

# 0n supprime l'index de chaque phrase qui est inutile au FCM
for i in range(0, len(lines), 3):        # Toutes les 3 lignes... donc chaque phrase
    line_split = lines[i].split()
    if(len(lines[i]) > 0):
        del line_split[0]
    lines[i] = ' '.join(line_split) + '\n'
    i+=1

#### Extraction de la première ligne du format d'entrée du FCM et ajout de la phrase nettoyée
<br>
Input:

    The <e1> child </e1> was carefully wrapped and bound into the <e2> cradle </e2> .
    Other
Output:

    Other	1	1	child	9	9	cradle	
    The child was carefully wrapped and bound into the cradle by means of a cord.

In [8]:
# On extrait la première ligne pour chaque phrase
# elle contient : la relation, les entités et leurs indices de début et fin
# On rajoute la phrase nettoyée en dessous, c'est le 1er resultat temporaire

lines_res_temp_1 = copy.deepcopy(lines)               # contiendra le 1er resultat temporaire
i = 0
for i in range(0, len(lines), 3):                     # Toutes les 3 lignes... donc chaque phrase
    line_split = lines[i].split()
    first_line = lines[i+1].replace('\n', '') + '	' # On met d'abord la relation dans first_line
    
    '''rel_sens = ''                                     # On extrait le sens de la relation
    j = 0                                                # (e1,e2) si e1 est avant e2 et vice-versa
    for j in range(len(line_split)):                     # à décommenter si besoin
        if(line_split[j] == '<e1>'):
            rel_sens = '(e1,e2)'
            break
        elif(line_split[j] == '<e2>'):
            rel_sens = '(e2,e1)'
            break
        j+=1
    first_line += rel_sens + '	'
    '''
    tabulation_fin = False                            # Boléen pour ne pas tabuler après e2
    j = 0
    while j < len(line_split):
        # Extraction des indices et suppresion des balises
        if('<e' in line_split[j]):
            tabulation_fin = not tabulation_fin
            del line_split[j]
            first_line += str(j) + '	'             # j = indice debut entité
            k = j                                     # k sert à parcourir l'entité commençant à j
            ent = ''
            while "</e" not in line_split[k]:
                ent += line_split[k]                  # On ajoute le mot de l'entité à ent
                if("</e" not in line_split[k+1]):     # Si entité non finie on espace
                    ent += ' '
                k+=1
            if(tabulation_fin == True):               # Si c'est la première entité on tabule
                ent += '	'
            
        elif('</e' in line_split[j]):
            del line_split[j]
            j-=1
            first_line += str(j) + '	' + ent       # j = indice fin entité
        
        # On construit le résultat temporaire 1, constitué des blocs vus plus haut
        lines_res_temp_1[i] = first_line + '\n'
        lines_res_temp_1[i+1] = ' '.join(line_split) + '\n'
        j+=1

#### Préparation du fichier à donner au SST
#### Puis lancement du SST
<br>
Input:
    
    Other	1	1	child	9	9	cradle	
    The child ...
Output:

    The	DT	the	0	0	0	child	NN	child	B-noun.person	B-E:PER_DESC	0	...

In [9]:
file = open("../sst/DATA/to_sst.txt", "w")              # Toutes les 3 lignes... donc chaque phrase
i = 0
file.write("	")                                   # Ne pas supprimer cette tabulation
# Préparation du fichier à donner au SST
# Uniquement les phrases, la première ligne servira plus tard
for i in range(1, len(lines_res_temp_1)+1, 3):
        file.write(lines_res_temp_1[i])
        file.write('\n')
        file.write('\n')
file.close()

# Commande pour lancer le SST, voir son README pour son fonctionnement si besoin de la modifier
command = ['sst', 'multitag-line', './DATA/to_sst.txt', '0', '0', 'DATA/GAZ/gazlistall_minussemcor',
          './MODELS/WSJPOSc_base_20', 'DATA/WSJPOSc.TAGSET',
           './MODELS/SEM07_base_12', './DATA/WNSS_07.TAGSET',
           './MODELS/WSJc_base_20', './DATA/WSJ.TAGSET',
           './MODELS/CONLL03_base_15', './DATA/CONLL03.TAGSET',
           '>', './DATA/res_sst.tags', '&', 'clean.bat']
process = subprocess.Popen(command, cwd="../sst", shell=True, stdout=subprocess.PIPE)
process.wait()   # On a attend la terminaison du SST, résultats dans res_sst.tags

0

#### Calcul du dernier tag pour chque mot
#### Suppression des lemmas
<br>
Input:
    
    The	DT	the	0	0	0	child	NN	child	B-noun.person	B-E:PER_DESC	0	...
Output:

    The	DT	0	0	0	child	NN	B-noun.person	B-E:PER_DESC	1	...
Note : Ici j'ai arbitrairement mis à 1 le dernier tag du mot 'child' pour l'exemple

In [10]:
# Lecture du fichier contenant les résultats du SST
# Toutes les 3 lignes, une phrase annotée
res_sst = open("../sst/DATA/res_sst.tags", "r")
lines_res_sst = res_sst.readlines()
res_sst.close()

# Génération de la dernière valeur du format de Yu pour chaque mot
# Pour chaque mot, on regarde si il se trouve sur le dependency path entre e1 et e2 dans sa phrase
# -> Génération d'un graphe de dépendances non orienté pour la phrase et chercher le plus court chemin
# si oui = '1' si non = '0'
for i in range(0, len(lines_res_sst), 3):
    line_split_res_sst = lines_res_sst[i].split()         # La ligne que l'on va modifier
    line_split_res_temp_1 = lines_res_temp_1[i].split()   # La phrase sans annotation pour le graphe
    
    # On trouve e1, sa taille et son indice de début
    e1_size = int(line_split_res_temp_1[2]) - int(line_split_res_temp_1[1])+1
    e1_start = int(line_split_res_temp_1[1])
    e1 = lines_res_temp_1[i+1].split()[e1_start]
    # Idem pour e2
    e2_size = int(line_split_res_temp_1[4+e1_size]) - int(line_split_res_temp_1[3+e1_size])+1
    e2_start = int(line_split_res_temp_1[3+e1_size])
    e2 = lines_res_temp_1[i+1].split()[e2_start]
    
    # On charge la phrase pour être traitée avec NetworkX
    doc = nlp(lines_res_temp_1[i+1])
    # On trouve le chemin de dépendance le plus court entre e1 et e2
    dep_path = shortest_dependency_path(doc, e1, e2)

    # Pour chaque mot..
    for j in range(5, len(line_split_res_sst), 6):
        # Si il est sur le chemin de dep et différent de e1 et e2
        if(line_split_res_sst[j-5] in dep_path
          and line_split_res_sst[j-5] != e1
          and line_split_res_sst[j-5] != e2):
            # Alors on met son dernier tag à 1
            line_split_res_sst[j] = '1'
        else:
            line_split_res_sst[j] = '0'
    lines_res_sst[i] = '	'.join(line_split_res_sst) + '\n'    
    
# Supression des lemmas
i = 0
for i in range(0, len(lines_res_sst), 3):                # Toutes les 3 lignes... donc chaque phrase
    line_split = lines_res_sst[i].split()
    
    j = 2
    while j < len(line_split):
        del line_split[j]                                # On supprime le lemma
        j+=5                                             # On avance au suivant
    lines_res_sst[i] = '	'.join(line_split) + '\n'

'''file = open("sst/DATA/res_sst.tags", "w")
for line in lines_res_sst:
    file.write(line)
file.close()'''

'file = open("sst/DATA/res_sst.tags", "w")\nfor line in lines_res_sst:\n    file.write(line)\nfile.close()'

#### Dans le résultat intermediaire 1 on remplace la phrase par sa version annotée et on prépare les deux lignes vides pour le res final
<br>
Input:
    
    Other	1	1	child	9	9	cradle	
et

    The	DT	0	0	0	child	NN	B-noun.person	B-E:PER_DESC	1	...
Output (résultat final):

    Other	1	1	child	9	9	cradle
    The	DT	0	0	0	child	NN	B-noun.person	B-E:PER_DESC	1
    1	B-noun.person B-E:PER_DESC
    1	B-noun.artifact 0

In [11]:
file = open("notebook_output.txt", "w")

# Dans le résultat final...
for i in range(len(lines_res_temp_1)):
    if(i%3 == 0):
        # On écrit la premiere ligne avec la relation, les entités et leurs indices
        file.write(lines_res_temp_1[i])
        # On ajoute sur la ligne suivante la phrase annotée avec le SST
        file.write(lines_res_sst[i])
        
        # On split ces deux lignes pour en extraire des informations
        # nécessaires à la construction des deux derniers blocs 
        line_split = lines_res_temp_1[i].split()
        sst_split = lines_res_sst[i].split()
        
        # On trouve la taille de e1, son indice de départ
        e1_size = int(line_split[2]) - int(line_split[1])+1
        e1_start = int(line_split[1])
        append1 = ''                        # append1 contiendra le 3ème bloc (celui décrivant e1)
        for j in range(e1_size):
            if(j > 0):
                append1 +=' '
            # On obtient les tags pour e1 (le 2ème et 3ème)
            append1 += sst_split[(e1_start+j)*5+2] + ' ' + sst_split[(e1_start+j)*5+3]
        file.write(str(e1_size) + '	' + append1 + '\n')
        
        # Idem pour e2
        e2_size = int(line_split[4+e1_size]) - int(line_split[3+e1_size])+1
        e2_start = int(line_split[3+e1_size])
        append2 = ''                        # append2 contiendra le 4ème bloc (celui décrivant e2)
        for j in range(e2_size):
            if(j > 0):
                append2 +=' '
            # On obtient les tags pour e2 (le 2ème et 3ème)
            append2 += sst_split[(e2_start+j)*5+2] + ' ' + sst_split[(e2_start+j)*5+3]
        file.write(str(e2_size) + '	' + append2 + '\n')

file.close()